# Fetal Health Classification using AI/ML E-13

### Importing inital libraries

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import joblib
import seaborn as sns

#library for data preprocessing & to read CSV files
import pandas as pd

# used for linear algebra and multi-dimensional matrix manipulation
import numpy as np 

import os

#used for data visualisation
import matplotlib.pyplot as plt

#matplotlib is used internally, easier to use
import seaborn as sb

### Loading and viewing the dataset

In [ ]:
data_frame = pd.read_csv("./fetal_health.csv")
data_frame.head()

# About the data

This dataset records of features extracted from Cardiotocogram (**CTG**) exams, which were then classified by expert *obstetrician* into 3 classes: 
- **Normal**
- **Suspect**
- **Pathological**
---
The Dataset has the following features/attributes:

- **baseline value**: Baseline Fetal Heart Rate (FHR) (beats per minute)
- **accelerations**: Number of accelerations per second
- **fetal_movement**: Number of fetal movements per second
- **uterine_contractions**: Number of uterine contractions per second
- **light_decelerations**: Number of light decelerations (LDs) per second
- **severe_decelerations**: Number of severe decelerations (SDs) per second
- **prolongued_decelerations**: Number of prolonged decelerations (PDs) per second
- **abnormal_short_term_variability**: Percentage of time with abnormal short term variability
- **mean_value_of_short_term_variability**: Mean value of short term variability
- **percentage_of_time_with_abnormal_long_term_variability**: Percentage of time with abnormal long term variability
- **mean_value_of_long_term_variability**: Mean value of long term variability
- **histogram_width**: Width of histogram made using all values from a record
- **histogram_min**: Histogram minimum value
- **histogram_max**: Histogram maximum value
- **histogram_number_of_peaks**: Number of peaks in the exam histogram
- **histogram_number_of_zeroes**: Number of zeros in the exam histogram
- **histogram_mode**: Histogram mode
- **histogram_mean**: Histogram mean
- **histogram_median**: Histogram median
- **histogram_variance**: Histogram variance
- **histogram_tendency**: Histogram tendency
- **fetal_health**: Encoded as 1-Normal; 2-Suspect; 3-Pathological.

### Retreiving information of the dataset 

In [ ]:
data_frame.info()

## DATA ANALYISIS & PRE-PROCESSING 

### Statistical trends in the data

In [ ]:
data_frame.describe().T

### Evaluating if our data is imbalanaced or not by analysing the target output.

In [ ]:
colours=["green","gold", "red"]
sb.countplot(data=data_frame, x="fetal_health",palette=colours)

#### We can see there is an imbalance in the data.

### A Heatmap to analyze how related any two datapoints are (Co-relation Matrix)

In [ ]:
correlation_matrix = data_frame.corr()
plt.figure(figsize=(15,15))
sb.heatmap(correlation_matrix, annot=True,center=0)
plt.savefig("co-relation.svg")

Based on the co-relation matrix we can see that
1. prolonged_decelerations
2. percentage of time with abnormal long term variabilty
3. mean value of long term variability

have higher co-relation with fetal health and hence these are the most important features that wil be essential in the prediction of the state of the fetus

In [ ]:
cols=['baseline value', 'accelerations', 'fetal_movement',
       'uterine_contractions', 'light_decelerations', 'severe_decelerations',
       'prolongued_decelerations', 'abnormal_short_term_variability',
       'mean_value_of_short_term_variability',
       'percentage_of_time_with_abnormal_long_term_variability',
       'mean_value_of_long_term_variability']
for i in cols:
#     sb.swarmplot(x=data_frame["fetal_health"], y=data_frame[i], color="black", alpha=0.5,size=1.5)
#     sb.boxenplot(x=data_frame["fetal_health"], y=data_frame[i], palette=colours)
    plt.show()

In [ ]:
# remove the target data from the dataset to visualise the the distribution of the attributtes
XData =data_frame.drop(["fetal_health"], axis=1)
# store the target data for future reference
YData = data_frame["fetal_health"]

### Visualise the distribution of data BEFORE feature scaling

In [ ]:
colors=["#483D8B","#4682B4", "#87CEFA"]
features=['baseline value', 'accelerations', 'fetal_movement','uterine_contractions', 'light_decelerations', 'severe_decelerations',
           'prolongued_decelerations', 'abnormal_short_term_variability', 'mean_value_of_short_term_variability',
             'percentage_of_time_with_abnormal_long_term_variability', 'mean_value_of_long_term_variability']
plt.figure(figsize=(20,10))
sb.boxenplot(data = XData,palette = colors)
plt.xticks(rotation=60)
plt.show()
plt.savefig("before-feature-scaling.svg")

### Visualise the distribution of data AFTER feature scaling

Import the standard scaler from the scikit-learn library to scale the data/features to make the data suitable for model training

In [ ]:
# Performing feature scaling
from sklearn.preprocessing import StandardScaler
standard_scaler = StandardScaler()
attributes = list(XData.columns)
s_scaler = StandardScaler()
XData_scaled= s_scaler.fit_transform(XData)
XData_scaled = pd.DataFrame(XData_scaled, columns=attributes)   
XData_scaled.describe().T
# Data after feature scaling


In [ ]:
plt.figure(figsize=(20,10))
sb.boxenplot(data = XData_scaled,palette = colors)
plt.xticks(rotation=60)
plt.show()
plt.savefig("after-feature-scaling.svg")

### Key Points
1. We can see from the above plot that the data is in the same range after the feature scaling
2. We can see some outliers in some attributes after the feature scaling
3. These outliers are not data entry errors or measurement errors as this data has been extracted from **CTG** data
4. Hence the outliers cannot be eliminated or dropped of from the model training as this will result in loss of information, and overfitting of the model (*fits exactly against training data.*)

## Model Building (YTBD)


### Splitting the data into train and test datasets (yet to be done) - First Review

In [ ]:
from sklearn.model_selection import train_test_split
XDataTrain, XDataTest, YTrain,YTest = train_test_split(XData_scaled,YData,test_size=0.2,random_state=25)

In [ ]:
from sklearn.model_selection import cross_val_score
from sklearn.metrics import precision_score, recall_score, confusion_matrix, classification_report, accuracy_score, f1_score
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve, auc, roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

#Training Linear Regression Classifier Model

LinearRegressionModel = Pipeline([('lr_classifier',LogisticRegression(random_state=25))])
LinearRegressionModel.fit(XDataTrain.values, YTrain.values)
filename = "./major/predict/LinearRegressionModel.sav"
joblib.dump(LinearRegressionModel,filename)
cv_score = cross_val_score(LinearRegressionModel, XDataTrain,YTrain, cv=12)
print(f"LinearRegression - {cv_score.mean()}")

In [ ]:
linear_pred = LinearRegressionModel.predict(XDataTest)
print(classification_report(YTest,linear_pred))

In [ ]:
DecisionTreeClassifierModel = Pipeline([('dt_classifier',DecisionTreeClassifier(random_state=25))])
DecisionTreeClassifierModel.fit(XDataTrain.values, YTrain)
filename = "./major/predict/DecisionTreeClassifierModel.sav"
joblib.dump(DecisionTreeClassifierModel,filename)
cv_score = cross_val_score(DecisionTreeClassifierModel, XDataTrain,YTrain, cv=12)
print(f"DecisionTreeClassifier - {cv_score.mean()}")

In [ ]:
decisionTree_pred = DecisionTreeClassifierModel.predict(XDataTest)
print(classification_report(YTest,decisionTree_pred))

In [ ]:
RandomForestClassifierModel = Pipeline([('rf_classifier',RandomForestClassifier())])
RandomForestClassifierModel.fit(XDataTrain, YTrain)
filename = "./major/predict/RandomForestClassifierModel.sav"
joblib.dump(RandomForestClassifierModel,filename)
cv_score = cross_val_score(RandomForestClassifierModel, XDataTrain,YTrain, cv=12)
print(f"RandomForestClassifier - {cv_score.mean()}")

In [ ]:
randomForest_pred = RandomForestClassifierModel.predict(XDataTest)
print(classification_report(YTest,randomForest_pred))

In [ ]:
SVCModel = Pipeline([('sv_classifier',SVC())])
SVCModel.fit(XDataTrain, YTrain)
filename = "./major/predict/SVCModel.sav"
joblib.dump(SVCModel,filename)
cv_score = cross_val_score(SVCModel, XDataTrain,YTrain, cv=12)
print(f"SVC - {cv_score.mean()}")

In [ ]:
svc_pred = SVCModel.predict(XDataTest)
print(classification_report(YTest,svc_pred))

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier
GradientBoostingClassifierModel = Pipeline([('gbcl_classifier',GradientBoostingClassifier())])
GradientBoostingClassifierModel.fit(XDataTrain, YTrain)
filename = "./major/predict/GradientBoostingClassifierModel.sav"
joblib.dump(GradientBoostingClassifierModel,filename)
cv_score = cross_val_score(GradientBoostingClassifierModel, XDataTrain,YTrain, cv=12)
print(f"GradientBoostingClassifierModel - {cv_score.mean()}")

In [ ]:
gradient_pred = GradientBoostingClassifierModel.predict(XDataTest)
print(classification_report(YTest,gradient_pred))

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
KNeighborsClassifierModel = Pipeline([('knn_classifier',KNeighborsClassifier())])
KNeighborsClassifierModel.fit(XDataTrain, YTrain)
filename = "./major/predict/KNeighborsClassifierModel.sav"
joblib.dump(KNeighborsClassifierModel,filename)
cv_score = cross_val_score(KNeighborsClassifierModel, XDataTrain,YTrain, cv=12)
print(f"KNeighborsClassifierModel - {cv_score.mean()}")

In [ ]:
knn_pred = KNeighborsClassifierModel.predict(XDataTest)
print(classification_report(YTest,knn_pred))

In [ ]:
# to tune the model we are choosing the hyperparameter values 
# random forest hypertune
# parameters = { 
#     'n_estimators': [100,150, 200,500,700,900],
#     'max_features': ['auto', 'sqrt', 'log2'],
#     'max_depth' : [4,6,8,12,14,16],
#     'criterion' :['gini', 'entropy'],
#     'n_jobs':[-1,1,None]
# }
# RandomForestClassifierModelHyperTuned = GridSearchCV(estimator=RandomForestClassifier(), param_grid=parameters, cv= 5)
# RandomForestClassifierModelHyperTuned.fit(XDataTrain, YTrain)
# filename = "./major/predict/RandomForestClassifierModelHyperTuned.sav"
# joblib.dump(RandomForestClassifierModelHyperTuned,filename)
# PredGBCLModelTrain = RandomForestClassifierModelHyperTuned.predict(XDataTest)

# gradient boost hyperparams
# params = {"loss": ["deviance"],
#               "learning_rate": [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1], 
#               "n_estimators": [200, 350, 500, 750],
#               "max_depth": [3, 6, 8]
#               }
# params = {"loss": ["log_loss"],
#               "learning_rate": [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1], 
#               "n_estimators": [200, 350, 500, 750],
#               "max_depth": [3, 6, 8]
#               }

In [ ]:
# RFCLModelTrain = RandomForestClassifierModelHyperTuned.score(XDataTrain,YTrain)
# RFCLModelTest = RandomForestClassifierModelHyperTuned.score(XDataTest,YTest)

# print(f"r^2(coeff of determination) on train set = {round(RFCLModelTrain, 3)}")
# print(f"r^2(coeff of determination) on test set = {round(RFCLModelTest, 3)}")

In [ ]:
GBCLModelTrain = GradientBoostingClassifierModel.score(XDataTrain,YTrain)
GBCLModelTest = GradientBoostingClassifierModel.score(XDataTest,YTest)
cv_score = cross_val_score(GradientBoostingClassifierModel, XDataTrain,YTrain, cv=12)
print(f"GradientBoostingClassifierModel - {cv_score.mean()}")

print(f"r^2(coeff of determination) on train set = {round(GBCLModelTrain, 3)}")
print(f"r^2(coeff of determination) on test set = {round(GBCLModelTest, 3)}")

In [ ]:
# RandomForestClassifierModelHyperTuned
# best_params = RandomForestClassifierModelHyperTuned.best_params_ # {'criterion': 'entropy', 'max_depth': 14, 'max_features': 'auto', 'n_estimators': 100, 'n_jobs': None}
# print(best_params)

# RF_model = RandomForestClassifier(**RandomForestClassifierModelHyperTuned.best_params_)
RF_model = RandomForestClassifier(n_estimators= 100,criterion='entropy', max_depth=14, max_features= 'auto',  n_jobs=None)
RF_model.fit(XDataTrain, YTrain)

cv_score = cross_val_score(RF_model, XDataTrain,YTrain, cv=12)
print(f"RandomForestClassifierModelHyperTuned - {cv_score.mean()}")
RFCLModelTrain = RF_model.score(XDataTrain,YTrain)
RFCLModelTest = RF_model.score(XDataTest,YTest)
filename = "./major/predict/RandomForestClassifierModelHyperTuned.sav"
joblib.dump(RF_model,filename)

print(f"r^2(coeff of determination) on train set = {round(RFCLModelTrain, 3)}")
print(f"r^2(coeff of determination) on test set = {round(RFCLModelTest, 3)}")

In [ ]:
pred_gbcl = GradientBoostingClassifierModel.predict(XDataTest)
acc = accuracy_score(YTest,pred_gbcl)
print(f" Testing Score of the model is {acc}")

In [ ]:
pred_gbcl = RF_model.predict(XDataTest)
acc = accuracy_score(YTest,pred_gbcl)
print(f" Testing Score of the model is {acc}")

In [ ]:
print(classification_report(YTest, pred_gbcl))

In [ ]:
import seaborn as sns
plt.subplots(figsize=(12,8))
cf_matrix = confusion_matrix(YTest, pred_gbcl)
sns.heatmap(cf_matrix/np.sum(cf_matrix), cmap=sns.diverging_palette(250, 10, s=80, l=55, n=9, as_cmap=True),annot = True, annot_kws = {'size':20})
plt.xlabel("Actual")
plt.ylabel("Predicted")
plt.show()